In [ ]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 13.8 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import SimpleITK as sitk
from PIL import Image
import nibabel as nib

In [ ]:
def register_images(ref_image_path, moving_image_path, output_path):
    # Load the fixed and moving images
    ref_image = cv2.imread(ref_image_path)  # Load as color
    moving_image = cv2.imread(moving_image_path)  # Load as color

    # Convert the images to grayscale for optical flow calculation
    ref_image_gray = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
    moving_image_gray = cv2.cvtColor(moving_image, cv2.COLOR_BGR2GRAY)

    # Calculate the optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(ref_image_gray, moving_image_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Warp the moving image using the flow field
    rows, cols, _ = ref_image.shape
    warped_image = np.zeros_like(ref_image)
    for i in range(rows):
        for j in range(cols):
            dx, dy = flow[i, j]

            # Avoid out-of-bounds indices
            warped_i = min(max(int(i+dy), 0), rows-1)
            warped_j = min(max(int(j+dx), 0), cols-1)

            warped_image[warped_i, warped_j] = moving_image[i, j]

    # Save the registered image to the output path
    cv2.imwrite(output_path, warped_image)

In [ ]:
def register(ref_image_path, moving_image_path, output_path):
    # Load the fixed and moving images
    ref_image = cv2.imread(ref_image_path)  # Load as color
    moving_image = cv2.imread(moving_image_path)  # Load as color

    # Convert the images to grayscale for optical flow calculation
    img1 = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(moving_image, cv2.COLOR_BGR2GRAY)

    # Warp the moving image using the flow field
    height,width = img1.shape

    orb_detector = cv2.ORB_create(800)

    # Extract key points and descriptors for both images
    kp1, des1 = orb_detector.detectAndCompute(img2, None)
    kp2, des2 = orb_detector.detectAndCompute(img1, None)

    # Display keypoints for reference image in green color
    imgKp_Ref = cv2.drawKeypoints(img1, kp1, 0, (0,222,0), None)
    cv2.imwrite('/content/keypoints.jpg', imgKp_Ref)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match the two sets of descriptors.
    matches = matcher.match(des1, des2)

    # Sort matches on the basis of their Hamming distance.
    matches = sorted(matches, key=lambda x: x.distance)

    # Take the top 90 % matches forward.
    matches = matches[:int(len(matches) * 0.9)]
    no_of_matches = len(matches)
    print(len(matches))

    # Display only 100 best matches {good[:100}
    imgMatch = cv2.drawMatches(img2, kp2, img1, kp1, matches[:100], None, flags = 2)
    cv2.imwrite('/content/matches.jpg', imgMatch)

    # Define empty matrices of shape no_of_matches * 2.
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
      p1[i, :] = kp1[matches[i].queryIdx].pt
      p2[i, :] = kp2[matches[i].trainIdx].pt

    # Find the homography matrix.
    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

    # Use this matrix to transform the
    # colored image wrt the reference image.
    warped_img = cv2.warpPerspective(moving_image,
                    homography, (width, height))

    # Save the registered image to the output path
    cv2.imwrite(output_path, warped_img)

In [ ]:
# Usage example
fixed_image_path = '/content/AEB1.jpg'
moving_image_path = '/content/AEB0.jpg'
output_path = '/content/aligned.jpg'

# register_images(fixed_image_path, moving_image_path, output_path)
register(fixed_image_path, moving_image_path, output_path)

216


In [ ]:
import cv2
import numpy as np

def opacity_analysis(image1_path, image2_path, output_path):
    # Load the images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Split the images into channels
    b1, g1, r1 = cv2.split(image1)
    b2, g2, r2 = cv2.split(image2)

    # Subtract the blue and green channels
    bg_diff = cv2.subtract(b1, b2)
    gg_diff = cv2.subtract(g1, g2)

    # Create a new image with blue and green channels
    result = cv2.merge((bg_diff, gg_diff, np.zeros_like(r1)))

    # Save the resulting image
    cv2.imwrite(output_path, result)


# Usage example
image1_path = '/content/AEB1.jpg'
image2_path = '/content/aligned.jpg'
output_path = '/content/outputopacity.jpg'
opacity_analysis(image1_path, image2_path, output_path)